# LLC4320 Examples

In [ ]:
import xarray as xr
import intake
import numpy as np
%matplotlib inline
import holoviews as hv
from holoviews.operation.datashader import regrid
from xmitgcm import llcreader
hv.extension('bokeh')

### Load Data

Data is stored in [zarr](http://zarr.readthedocs.io) format on Google Cloud Storage.
This format is optimized for fast parallel access in the cloud.

In [ ]:
cat_url = "https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/llc4320.yaml"
cat = intake.Catalog(cat_url)
sst = cat.LLC4320_SST.to_dask()
u = cat.LLC4320_SSU.to_dask()
v =  cat.LLC4320_SSV.to_dask()

We merge the variables and convert from the 13-face LLC layout to a regular rectangular grid (excluding the arctic).

In [ ]:
ds = xr.merge([sst, u, v])
ds = llcreader.llcmodel.faces_dataset_to_latlon(ds, metric_vector_pairs=[])
ds

In [ ]:
coords = cat.LLC4320_grid.to_dask().reset_coords()
coords = llcreader.llcmodel.faces_dataset_to_latlon(coords)
coords

In [ ]:
print(f'Dataset Total Size: {ds.nbytes / 1e12:3.1f} TB')

This is a huge dataset, and that's only one level out of 90!

### Launch Dask Cluster

This allows us to parallelize calculations across cloud computing nodes.

In [ ]:
from dask_kubernetes import KubeCluster
from dask.distributed import Client
cluster = KubeCluster()
cluster.adapt(minimum=1, maximum=20)
client = Client(cluster)
cluster

### Sea Surface Temperature

An interactive visualization.

In [ ]:
ocean_mask = coords.hFacC.reset_coords(drop=True)>0
sst = (ds.SST.where(ocean_mask).rename('SST'))
hv_image = hv.Dataset(sst).to(hv.Image, kdims=['i', 'j'], dynamic=True)

%output holomap='scrubber' fps=1
%opts Image [width=900 height=500 colorbar=True bgcolor='gray'] (cmap='RdBu_r')
regrid(hv_image, precompute=True)

### Use XGCM to Perform Calculus

[XGCM](https://xgcm.readthedocs.io/en/latest/) is a python packge for working with the datasets produced by numerical General Circulation Models (GCMs) and similar gridded datasets that are amenable to finite volume analysis. In these datasets, different variables are located at different positions with respect to a volume or area element (e.g. cell center, cell face, etc.) xgcm solves the problem of how to interpolate and difference these variables from one position to another.

In [ ]:
import xgcm
grid = xgcm.Grid(coords.drop(['k', 'k_p1']), periodic=['X'])
grid

### Vorticity

An interactive map of relative vorticity.

In [ ]:
zeta = (-grid.diff(ds.U * coords.dxC, 'Y', boundary='extend')
        +grid.diff(ds.V * coords.dyC, 'X', boundary='extend'))/coords.rAz
zeta

In [ ]:
vort_image = hv.Dataset(zeta.rename('vort')).to(hv.Image, kdims=['i_g', 'j_g'], dynamic=True)

%output holomap='scrubber' fps=0.25
%opts Image [width=900 height=500 colorbar=True bgcolor='gray' logz=False] (cmap='RdBu_r')
regrid(vort_image, precompute=True).redim.range(vort=(-1e-4, 1e-4))

### Kinetic Energy

In [ ]:
eke = 0.5 * ( grid.interp(ds.U, 'X', boundary='fill')**2 +
              grid.interp(ds.V, 'Y', boundary='fill')**2
            ).where(ocean_mask).rename('EKE')
eke

In [ ]:
eke_image = hv.Dataset(np.log10(eke)).to(hv.Image, kdims=['i', 'j'], dynamic=True)

%output holomap='scrubber' fps=0.25
%opts Image [width=900 height=500 colorbar=True bgcolor='gray' logz=False] (cmap='magma')
rg = regrid(eke_image, precompute=True).redim.range(EKE=(-4, 0))
rg

## Daily-Averaged Kinetic Energy

The data is resampled on the fly.

In [ ]:
ds_daily = ds.resample(time='D').mean()
ds_daily

In [ ]:
eke_daily = 0.5 * (grid.interp(ds_daily.U, 'X', boundary='fill')**2 +
                   grid.interp(ds_daily.V, 'Y', boundary='fill')**2
                  ).where(ocean_mask).rename('EKE')
eke_daily

In [ ]:
eke_image = hv.Dataset(np.log10(eke_daily)).to(hv.Image, kdims=['i', 'j'], dynamic=True)

%output holomap='scrubber' fps=0.25
%opts Image [width=800 height=500 colorbar=True bgcolor='gray' logz=False] (cmap='magma')
regrid(eke_image, precompute=True).redim.range(EKE=(-4, 0))